# Jupyter notebook based on ImageD11 to process 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 21/02/2025

In [ ]:
import os

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())

In [ ]:
# this cell is tagged with 'parameters'
# to view the tag, select the cell, then find the settings gear icon (right or left sidebar) and look for Cell Tags

PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

dset_path = ''

phase_str = 'Fe'

# path to parameters .json/.par
parfile = ''

# peak filtration options
cf_strong_frac = 0.991
cf_strong_dsmax = 1.01
cf_strong_dstol = 0.01

# friedel pair search options
womega = 1.0
weta = 1.0
wtth = 1.5
wI = 0.5

# indexing options
indexer_ds_tol = 0.003

rings_for_gen = [1, 3]

# now we want to decide which rings to score our found orientations against
# generally we can just exclude dodgy rings (close to other phases, only a few peaks in etc)
rings_for_scoring = [0, 1, 2, 3]

# the sequence of hkl tolerances the indexer will iterate through
hkl_tols_seq = [0.01, 0.02]
# the sequence of minpks fractions the indexer will iterate through
fracs = [0.9, 0.6]

# the max number of UBIs we can find per pair of rings
max_grains = 1000

# makemap refinement options
symmetry = "cubic"

gridpars = {
        'DSTOL' : 0.004,
        'NUL' : True,
        'FITPOS' : True,
        'tolangle' : 0.25,
        'toldist' : 100.,
        'NTHREAD' : 1 ,
        'NPKS': 25
}

absolute_minpks = 25

dset_prefix = 'ff'

In [ ]:
# import functions we need

import os, glob, pprint
import numpy as np
import h5py
from tqdm.notebook import tqdm

import matplotlib
%matplotlib widget
from matplotlib import pyplot as plt
import scipy.spatial

# import utils
from ImageD11.nbGui import nb_utils as utils

import ImageD11.grain
import ImageD11.indexing
import ImageD11.columnfile
import ImageD11.refinegrains
import ImageD11.grid_index_parallel
from ImageD11.sinograms import properties, dataset

from ImageD11.blobcorrector import eiger_spatial
from ImageD11.peakselect import select_ring_peaks_by_intensity, filter_peaks_by_phase

In [ ]:
# load the dataset from file

ds = ImageD11.sinograms.dataset.load(dset_path)

sample = ds.sample
dataset = ds.dset
rawdata_path = ds.dataroot
processed_data_root_dir = ds.analysisroot
print(ds)
print(ds.shape)

In [ ]:
ds.parfile = parfile
ds.save()

In [ ]:
ds.phases = ds.get_phases_from_disk()
ds.phases.unitcells

In [ ]:
# now let's select a phase to index from our parameters json

ucell = ds.phases.unitcells[phase_str]

print(ucell.lattice_parameters, ucell.spacegroup)

In [ ]:
# load 3d columnfile from disk

cf_3d = ds.get_cf_3d_from_disk()
ds.update_colfile_pars(cf_3d, phase_name=phase_str) 

cf_3d_path = f'{sample}_{dataset}_3d_peaks.flt'
cf_3d.writefile(cf_3d_path)

In [ ]:
# plot the 3D peaks (fewer of them) as a cake (two-theta vs eta)
# if the parameters in the par file are good, these should look like straight lines

ucell.makerings(cf_3d.ds.max())

fig, ax = plt.subplots(figsize=(16,9), layout='constrained')

ax.scatter(cf_3d.ds, cf_3d.eta, s=1)
ax.vlines(ucell.ringds, -50, 50, zorder=0, color='red')

ax.set_xlabel("D-star")
ax.set_ylabel("eta")

plt.show()

# First step: Visually inspect if we can easily see Friedel pairs
# Not worth doing if we can't see them!

In [ ]:
# here we are filtering our peaks (cf_3d) to select only the strong peaks
cf_strong = select_ring_peaks_by_intensity(cf_3d, frac=cf_strong_frac, dsmax=cf_strong_dsmax, doplot=0.8, dstol=cf_strong_dstol)
print(f"Got {cf_strong.nrows} strong peaks for indexing")

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9), constrained_layout=True)

ax.plot(cf_3d.ds, cf_3d.sum_intensity,',', label='cf_3d')
ax.plot(cf_strong.ds, cf_strong.sum_intensity,',', label='cf_strong')
ax.vlines(ucell.ringds, 1e3, 1e4, zorder=0, color='red')
ax.semilogy()

ax.set_xlabel("Dstar")
ax.set_ylabel("Intensity")
ax.legend()

plt.show()

In [ ]:
# compute Lorenz factor
lf = ImageD11.refinegrains.lf(cf_strong.tth, cf_strong.eta)

f = plt.figure(figsize=(15,5), layout='constrained')
ax = f.add_subplot()

# select peaks between 3 and 5 degrees in omega
om1 = (cf_strong.omega < 5) & (cf_strong.omega > 3)

# plot omega against intensity for those peaks, coloured by eta (azimuthal position on the ring)
ax.scatter(cf_strong.omega[om1], cf_strong.sum_intensity[om1], c=cf_strong.eta[om1], marker='o')

# the friedel pair of these peaks should be 180 degrees away in eta
etapair = 180 - cf_strong.eta

# modulate
etapair = np.where(etapair > 180, etapair - 360, etapair)

# select peaks for the friedel pairs between 183 and 185 degrees
om2 = (cf_strong.omega < 185) & (cf_strong.omega > 183)

# plot omega against intensity for the friedel pairs as crosses
ax.scatter(cf_strong.omega[om2] - 180, cf_strong.sum_intensity[om2], c=etapair[om2], marker='+')
ax.semilogy()

ax.set(xlabel='omega (deg)', ylabel='peak intensity', title='Coloured by eta')

# for valid friedel pairs, we should see 'o' and '+' markers close together in omega and intensity, with similar colours (eta)
plt.show()

In [ ]:
def calc_tth_eta( c, pi, pj ):
    dX = c.xl[pi] + c.xl[pj]
    dY = c.yl[pi] + c.yl[pj]
    dZ = c.zl[pi] - c.zl[pj]
    r = np.sqrt(dY*dY + dZ*dZ)
    tth = np.degrees( np.arctan2( r, dX )  )
    eta = np.degrees(np.arctan2( -dY, dZ ))
    return tth, eta

def find_friedel_pairs(cf_in, womega=1.5, weta=0.2, wtth=1.5, wI=0.5, doplot=False):
    # create a 4-dimensional tree
    # dimensions are omega, eta, tth, intensity
    t1 = scipy.spatial.cKDTree( np.transpose( [ 
                                womega*(cf_in.omega%360),
                                weta*(cf_in.eta%360),
                                wtth*cf_in.tth,
                                wI*np.log10(cf_in.sum_intensity) ] ))
    
    # create another tree for the friedel pair side (omega + 180, 180 - eta)
    t2 = scipy.spatial.cKDTree( np.transpose([ 
                                 womega*((cf_in.omega+180)%360),
                                 weta*((180-cf_in.eta)%360),
                                 wtth* cf_in.tth,
                                 wI*np.log10(cf_in.sum_intensity) ] ))
    
    # create a distance matrix between trees with a max distance of 1, returning a sparse matrix
    coo = t1.sparse_distance_matrix( t2, max_distance=1, output_type='coo_matrix' ) # 1 degree eta might be tight?
    
    inds = np.arange(cf_in.nrows)
    
    # mask for peaks on one side of the friedel pair
    p1 = inds[coo.row]
    # mask for peaks on the other side of the friedel pair
    p2 = inds[coo.col]
    
    # compute tth-eta of friedel pairs
    tth, eta = calc_tth_eta( cf_in, p1, p2 )
    # mask for intensity for friedel pairs
    s1 = cf_3d.sum_intensity[p1]
    s2 = cf_3d.sum_intensity[p2]
    
    # convert tth to dstar
    dstar = 2*np.sin(np.radians(tth)/2)/cf_in.parameters.get('wavelength')
    
    if doplot:
        f,a = plt.subplots(2,1,figsize=(20,6), layout='constrained', sharex=True)
        a[0].hist2d(dstar,eta,bins=(2000,360), norm='log', weights=s1+s2)
        a[0].plot(ucell.ringds, np.zeros_like(ucell.ringds),"|r",lw=1,ms=90)
        a[0].set(ylabel='eta (deg)')
        a[1].hist2d(dstar,coo.data,  # sum of squares of distance matrix
        #            np.log(s1+s2),
                    bins=(1000,128), norm='log');
        a[1].vlines(ucell.ringds, -50, 50, zorder=0, color='red')
        a[1].set(xlabel='dstar', ylabel='distance for search')
        f.suptitle(' Top: D-star vs eta of Friedel pairs \n Bottom: D-star vs 4D tree distance')
        plt.show()
    
    if doplot:
        f,a = plt.subplots(t1.data.shape[1],1,figsize=(20,6), layout='constrained', sharex=True)
        for i in range(t1.data.shape[1]):
            a[i].hist2d(dstar, t1.data[coo.row,i] - t2.data[coo.col,i], bins=(1000,128), norm='log')
            a[i].vlines(ucell.ringds, -0.1, 0.1, zorder=0, color='red')
            a[i].set(ylabel=['Omega', 'Eta', 'Two-theta', 'Log peak intensity'][i])
        f.suptitle('D-star vs error in [omega, eta, tth, intensity] for the friedel pair')
        f.supxlabel('dstar')
        plt.show()
    
    # Mask to powder rings
    m = np.zeros_like(p1, dtype=bool)
    for d in ucell.ringds:
        m |= abs(dstar - d)<0.002
    
    # make columnfiles for each side of the friedel pair
    c1 = cf_in.copyrows( p1[m] )
    c2 = cf_in.copyrows( p2[m] )
    
    c1.tth[:] = tth[m]
    c2.tth[:] = tth[m]
    c1.ds[:] = dstar[m]
    c2.ds[:] = dstar[m]
    
    if doplot:
        fig, ax = plt.subplots(layout='constrained')
        ax.plot(c1.eta%360, eta[m]%360,',')
        ax.set(xlabel='eta (deg)', ylabel='eta (deg)', title='Observed vs computed eta for Friedel pairs (c1)')
        plt.show()
    
    # computed eta values (variable eta) matches c1, so we take c1.eta as eta, then recompute c2.eta
    c1.eta[:] = eta[m]
    e2 = 180 - eta[m]
    c2.eta[:] = np.where( e2 > 180, e2-360, e2)
    
    # combine paired peaks into one columnfile
    cpair = ImageD11.columnfile.colfile_from_dict({
        t: np.concatenate( (c1[t], c2[t]) ) for t in c1.titles } )
    cpair.parameters = cf_in.parameters
    
    if doplot:
        fig, ax = plt.subplots(figsize=(20,6), layout='constrained')
        ax.plot(c1.ds, c1.eta, ',')
        ax.plot(c2.ds, c2.eta, ',')
        ax.plot(cpair.ds, cpair.eta, ',')
        ax.vlines(ucell.ringds, -50, 50, zorder=0, color='red')
        ax.xlabel('D-star')
        ax.ylabel('Eta')
        plt.show()
    
    cpair.gx[:],cpair.gy[:],cpair.gz[:] = ImageD11.transform.compute_g_vectors( cpair.tth, cpair.eta, cpair.omega, cpair.parameters.get('wavelength') )
    
    if doplot:
        fig, ax = plt.subplots(figsize=(20,6), layout='constrained')
        ax.plot(cpair.ds, cpair.sum_intensity*np.exp(5*cpair.ds**2),',')
        ax.semilogy()
        ax.vlines(ucell.ringds, 1e3, 1e4, zorder=0, color='red')
        ax.xlabel('D-star')
        ax.ylabel('Weighted Sum intensity')
        plt.show()
        
    return cpair

In [ ]:
cf_friedel_pairs = find_friedel_pairs(cf_strong, womega=womega, weta=weta, wtth=wtth, wI=wI, doplot=True)

In [ ]:
# plot the results

fig, ax = plt.subplots(layout='constrained', figsize=(10, 5))
ax.plot(cf_3d.ds, cf_3d.eta, ',', label='cf_3d')
ax.plot(cf_friedel_pairs.ds, cf_friedel_pairs.eta, ',', label='cf_friedel_pairs')
ax.vlines(ucell.ringds, -50, 50, zorder=0, color='red')
ax.set(xlabel='d-star', ylabel='eta')
ax.legend()
plt.show()

In [ ]:
# plot the results

fig, ax = plt.subplots(layout='constrained', figsize=(10, 5))
ax.plot(cf_3d.ds, cf_3d.sum_intensity, ',', label='cf_3d')
ax.plot(cf_friedel_pairs.ds, cf_friedel_pairs.sum_intensity, ',', label='cf_friedel_pairs')
ax.vlines(ucell.ringds, 1e3, 1e4, zorder=0, color='red')
ax.set(xlabel='d-star', ylabel='sum intensity')
ax.semilogy()
ax.legend()
plt.show()

In [ ]:
# Now we index the friedel pair resuls

# the tolerance in g-vector angle
cosine_tol = np.cos(np.radians(90 - ds.ostep))

_, indexer = utils.do_index(cf=cf_friedel_pairs,
                            dstol=indexer_ds_tol,
                            forgen=rings_for_gen,
                            foridx=rings_for_scoring,
                            hkl_tols=hkl_tols_seq,
                            fracs=fracs,
                            cosine_tol=cosine_tol,
                            max_grains=max_grains,
                            unitcell=ucell
)

In [ ]:
# inspect the results of the index

indexer.histogram_drlv_fit()

fig, ax = plt.subplots(layout='constrained', figsize=(10, 5))
for row in indexer.histogram:
    ax.plot(indexer.bins[1:-1], row[:-1],'-')

ax.set(xlabel='Peak error', ylabel='npeaks')
plt.show()

In [ ]:
# now we switch to grid indexing

In [ ]:
nproc = len(os.sched_getaffinity(os.getpid())) - 3

omega_slop = ds.ostep/2

gridpars['TOLSEQ'] = [hkl_tols_seq[0],]
gridpars['COSTOL'] = np.cos(np.radians(90 - ds.ostep))
gridpars['NPROC'] = nproc
gridpars['OMEGAFLOAT'] = omega_slop
gridpars['SYMMETRY'] = symmetry
gridpars['RING1'] = rings_for_gen
gridpars['RING2'] = rings_for_gen

# copy out the columnfile used by the indexer
cf_indexed = indexer.colfile.copy()

cf_indexed.addcolumn(indexer.ga.copy(), 'labels')
cf_indexed.addcolumn(np.zeros(cf_indexed.nrows), 'drlv2')

for v in 'xyz':
    cf_3d.parameters.stepsizes[f't_{v}'] = 0.1

fittedgrains = []
for i in range(len(indexer.ubis)):
    grains = [ImageD11.grain.grain(indexer.ubis[i].copy() ),]
    # only take indexed spots using Friedel pairs
    cfit = ImageD11.columnfile.colfile_from_dict(
        { t:cf_indexed[t][indexer.ga==i+1] for t in cf_indexed.titles} )
    if cfit.nrows == 0:
        continue
    fitted = ImageD11.grid_index_parallel.domap( cf_3d.parameters,
                                    cfit,
                                    grains,
                                    gridpars )
    if len(fitted) > 0:
        fittedgrains.append( fitted[0] )

In [ ]:
# 3D scatter plot of grain positions coloured by grain volume

utils.plot_grain_positions(fittedgrains, colour='npks', centre_plot=False, size_scaling=0.5)

In [ ]:
fig, ax = plt.subplots()
ax.hist([float(grain.npks) for grain in fittedgrains], bins=50)
plt.show()

In [ ]:
# filter out grains with fewer than absolute_minpks peaks
grains_filtered = [grain for grain in fittedgrains if float(grain.npks) > absolute_minpks]

In [ ]:
# 3D scatter plot of grain positions coloured by grain volume

utils.plot_grain_positions(grains_filtered, colour='npks', centre_plot=False, size_scaling=0.5)

In [ ]:
# write the filtered grains to disk

filtered_map_path = f'{sample}_{dataset}_nice_grains.map'

ImageD11.grain.write_grain_file(filtered_map_path, grains_filtered)

In [ ]:
# write cf_3d to disk temporarily

cf_3d_path = f'{sample}_{dataset}_3d_peaks.flt'
cf_3d.writefile(cf_3d_path)

In [ ]:
# now we write a classic parameter file for makemap.py
oldparfile = phase_str + '.par'
ds.phases.to_old_pars_file(oldparfile, phase_str)

In [ ]:
# run makemap again against all peaks

new_filtered_map_path = f'{sample}_{dataset}_nice_grains.map.new'
new_cf_3d_path = cf_3d_path + '.new'

final_makemap_tol = hkl_tols_seq[0]

makemap_output = !makemap.py -p {oldparfile} -u {filtered_map_path} -U {new_filtered_map_path} -f {cf_3d_path} -s {symmetry} -t {final_makemap_tol} --omega_slop={omega_slop} --no_sort

In [ ]:
grains_final = ImageD11.grain.read_grain_file(new_filtered_map_path)

# 3D scatter plot of grain positions coloured by grain volume

utils.plot_grain_positions(grains_final, colour='npks', centre_plot=False, size_scaling=0.5)

In [ ]:
utils.plot_grain_histograms(new_cf_3d_path, new_filtered_map_path, oldparfile, omega_slop, tol=hkl_tols_seq[0])

In [ ]:
unit_cell_lengths = np.array([np.sort(g.unitcell.copy()[:3]) for g in grains_final])
median_unit_cell = np.median(unit_cell_lengths, axis=0)

fig, ax = plt.subplots(constrained_layout=True)
ax.plot(unit_cell_lengths)
ax.hlines(median_unit_cell, 0, len(unit_cell_lengths))
ax.set_xlabel("Grain ID")
ax.set_ylabel("Unit cell length")
plt.show()

In [ ]:
# import makemap output columnfile with peak assignments
cf_3d = ImageD11.columnfile.columnfile(new_cf_3d_path)

# write 3D columnfile to disk
ImageD11.columnfile.colfile_to_hdf(cf_3d, ds.col3dfile, name='peaks')

# save grain data
ds.save_grains_to_disk(grains_final, phase_name=phase_str)

ds.save()

In [ ]:
# cleaning up

for path in [
    cf_3d_path,
    filtered_map_path,
    new_filtered_map_path,
    new_cf_3d_path,
]:
    if os.path.exists(path):
        os.remove(path)